In [2]:
"""
In order to run notebook, all files in '/content/drive/My Drive'
"""

import sys 
import os
sys.path.append(os.path.abspath("/content/drive/My Drive"))
from network import LeNet
from random import shuffle, choice
from run import train
from utils import *
import torch.nn.functional as F
import torch
import csv
import pandas as pd

In [ ]:
train_loader ,test_loader = load_dataset('http://deeplearning.net/data/mnist/mnist.pkl.gz',500, 'content/data')

In [ ]:
pools = ('max', 'avg')
# Activation functions

# activations = [
#                F.elu,
#                F.relu,
#                F.relu6,
#                F.rrelu,
#                F.leaky_relu,
#                F.selu,
#                F.celu,
#                F.gelu,
#                F.hardshrink,
#                F.hardtanh,
#                F.logsigmoid,
#                torch.sigmoid,
#                torch.tanh,
#                F.softshrink,
#                F.softsign
#                ]

# Best handpicked choice
activations = [
               F.elu,
               F.relu,
               F.rrelu,
               F.leaky_relu,
               F.celu
              ]



pools_list = combination(pools, 2)
activations_list = combination(activations, 5)
learning_rates = lr_decay(10,19)
learning_rates.append(0.018182)

same_activations_list = []
for act_tup in activations_list:
  if len(set(act_tup)) == 1:
    activations_list.remove(act_tup)
print(len(activations_list))

121


#### Effects of activation functions on fixed epoch and learning rates.

*Activation functions mentioned in LeNet i.e tanh is not included in here. This experimentation includes newly discovered activation functions.*

In [ ]:
"""
Default:
      batch_size: 500
"""
shuffle(activations_list)

results = {}

with open('/content/drive/My Drive/result.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(['Epoch', 'Learning_rate', 'Training_accuracy', 'Testing_accuracy', 'Activation','pool'])
  for rate in learning_rates:
    for act in activations_list: 
      activation = act
      pool = choice(pools_list)
      model, train_acc = train(train_loader, LeNet, epochs=10, lr=rate, use_cuda=True, pools=pool, activations=activation, print_every=False)
      test_acc = check_accuracy(test_loader , model, use_cuda=True)
      writer.writerow([10, rate, train_acc, test_acc, [ str(x.__name__)for x in activation], pool])
print("---Finished---")

Accuracy on test: 95.73
Accuracy on test: 93.32
Accuracy on test: 95.36
Accuracy on test: 72.66
Accuracy on test: 91.21
Accuracy on test: 83.09
Accuracy on test: 94.92
Accuracy on test: 90.85
Accuracy on test: 98.29
Accuracy on test: 44.32
Accuracy on test: 94.44
Accuracy on test: 80.7
Accuracy on test: 79.98
Accuracy on test: 59.79
Accuracy on test: 69.04
Accuracy on test: 98.06
Accuracy on test: 91.54
Accuracy on test: 73.11
Accuracy on test: 93.25
Accuracy on test: 68.53
Accuracy on test: 47.95
Accuracy on test: 98.65
Accuracy on test: 97.12
Accuracy on test: 69.61
Accuracy on test: 98.55
Accuracy on test: 98.35
Accuracy on test: 80.11
Accuracy on test: 72.12
Accuracy on test: 81.45
Accuracy on test: 24.77
Accuracy on test: 96.91
Accuracy on test: 83.76
Accuracy on test: 97.0
Accuracy on test: 98.56
Accuracy on test: 78.41
Accuracy on test: 97.72
Accuracy on test: 98.27
Accuracy on test: 87.31
Accuracy on test: 9.91
Accuracy on test: 84.99
Accuracy on test: 98.27
Accuracy on test: 8

In [4]:
results_and_epochs = pd.read_csv('/content/drive/My Drive/results.csv')
best_results = results_and_epochs[results_and_epochs['Testing_accuracy'] > 98][['Learning_rate', 'Training_accuracy', 'Testing_accuracy', 'Activation', 'pool']].sort_values(by='Testing_accuracy', ascending=True)
best_results

,Learning_rate,Training_accuracy,Testing_accuracy,Activation,pool
630,0.012500,97.7824,98.01,"['relu', 'relu', 'rrelu', 'rrelu', 'celu']","('max', 'avg')"
658,0.012500,97.5826,98.01,"['relu', 'relu', 'relu', 'rrelu', 'rrelu']","('max', 'max')"
236,0.016667,97.2150,98.02,"['relu', 'relu', 'rrelu', 'rrelu', 'rrelu']","('max', 'max')"
640,0.012500,97.7946,98.03,"['relu', 'relu', 'rrelu', 'celu', 'celu']","('avg', 'avg')"
129,0.016667,97.7642,98.04,"['relu', 'leaky_relu', 'leaky_relu', 'celu', '...","('max', 'max')"
...,...,...,...,...,...
554,0.013333,97.4002,98.87,"['relu', 'relu', 'relu', 'leaky_relu', 'leaky_...","('avg', 'avg')"
868,0.011111,97.4600,98.88,"['relu', 'relu', 'relu', 'relu', 'celu']","('max', 'avg')"
278,0.015385,97.2978,98.89,"['relu', 'relu', 'relu', 'relu', 'leaky_relu']","('avg', 'avg')"
1018,0.010526,97.7580,98.89,"['relu', 'relu', 'relu', 'rrelu', 'celu']","('max', 'avg')"


In [5]:
best_results.to_csv('/content/drive/My Drive/best_results.csv')